In [0]:
%pip install pyspark==3.2.0
%pip install geopandas

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 34.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805899 sha256=1afb9184b862718bfd03be1e6982767b20d320f7f44a03857ee2d47a954427fd
  Stored in directory: /root/.cache/pip/wheels/3d/f5/13/04a82efe56a577a8f1509e75ffd4253dc31ee18bae0ff701ea
Successfully built pyspark
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/

In [0]:
dbutils.library.restartPython()

## 1.1 Import data and Inspect
*    missing value/ duplication/ consistency
*    change data type for numerical and catagorical data

In [0]:
# Importing and inspect data --------------------------------------------------
import pyspark.pandas as ps
# s3_bucket_path = "s3://nyc-taxi-buk/dataset/nyc_taxi_trip_duration.csv"
#sdf = ps.read_csv(s3_bucket_path)

# Read the data into a DataFrame
sdf = ps.read_table("tripduration.default.raw_data")

# Inspect data --------------------------------------------------
print(sdf.info())
display(sdf.head())

# Inspect missing value issue --------------------------------------------------
print('total missing values: ',sdf.isnull().sum())
print('duplicated value: ', sdf.duplicated().sum()) # no duplicated value
    # there is 11 features with 729322 rows with no null and duplicate value 

# Change data types  --------------------------------------------------
sdf['id'] = sdf['id'].astype(str)
sdf['id'] = sdf['id'].str.replace('^id', '')
sdf['id'] = sdf['id'].astype('int64') # converting to integer
sdf['store_and_fwd_flag'] = sdf['store_and_fwd_flag'].astype('str') # converting to category
sdf['vendor_id'] = sdf['vendor_id'].astype('str') # converting to category 
sdf['pickup_datetime'] = ps.to_datetime(sdf['pickup_datetime'], format='%Y-%m-%d %H:%M:%S') # converting to datetime
sdf['dropoff_datetime'] = ps.to_datetime(sdf['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
sdf['trip_duration_min'] = (sdf['trip_duration']/60).astype(int)
sdf['trip_duration'] = ps.to_timedelta(sdf['trip_duration'], unit='s') # converting to timedelta     
print(sdf.info())
sdf.head()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  729322 non-null  object        
 1   vendor_id           729322 non-null  int64         
 2   pickup_datetime     729322 non-null  datetime64[ns]
 3   dropoff_datetime    729322 non-null  datetime64[ns]
 4   passenger_count     729322 non-null  int64         
 5   pickup_longitude    729322 non-null  float64       
 6   pickup_latitude     729322 non-null  float64       
 7   dropoff_longitude   729322 non-null  float64       
 8   dropoff_latitude    729322 non-null  float64       
 9   store_and_fwd_flag  729322 non-null  object        
 10  trip_duration       729322 non-null  int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(2)None


id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id1080784,2,2016-02-29T16:40:21Z,2016-02-29T16:47:01Z,1,-73.95391845703125,40.77887344360352,-73.96387481689453,40.77116394042969,N,400
id0889885,1,2016-03-11T23:35:37Z,2016-03-11T23:53:57Z,2,-73.98831176757811,40.73174285888672,-73.9947509765625,40.69493103027344,N,1100
id0857912,2,2016-02-21T17:59:33Z,2016-02-21T18:26:48Z,2,-73.997314453125,40.721458435058594,-73.94802856445312,40.774917602539055,N,1635
id3744273,2,2016-01-05T09:44:31Z,2016-01-05T10:03:32Z,6,-73.961669921875,40.75971984863281,-73.95677947998048,40.780628204345696,N,1141
id0232939,1,2016-02-17T06:42:23Z,2016-02-17T06:56:31Z,1,-74.01712036132812,40.70846939086913,-73.9881820678711,40.740631103515625,N,848


total missing values:  id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64
duplicated value:  0


/databricks/spark/python/pyspark/pandas/strings.py:1608: FutureWarning: Default value of `regex` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 729322 entries, 0 to 729321
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype      
---  ------              --------------   -----      
 0   id                  729322 non-null  int64      
 1   vendor_id           729322 non-null  <U0        
 2   pickup_datetime     729322 non-null  datetime64 
 3   dropoff_datetime    729322 non-null  datetime64 
 4   passenger_count     729322 non-null  int64      
 5   pickup_longitude    729322 non-null  float64    
 6   pickup_latitude     729322 non-null  float64    
 7   dropoff_longitude   729322 non-null  float64    
 8   dropoff_latitude    729322 non-null  float64    
 9   store_and_fwd_flag  729322 non-null  <U0        
 10  trip_duration       729322 non-null  timedelta64
 11  trip_duration_min   729322 non-null  int64      
dtypes: datetime64(2), float64(4), int64(3), str(2), timedelta64(1)None


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_duration_min
0,1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,0 days 00:06:40,6
1,889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,0 days 00:18:20,18
2,857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,0 days 00:27:15,27
3,3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,0 days 00:19:01,19
4,232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,0 days 00:14:08,14


## 1.2 Feature selection and engineering 
*    Extract datetime date
*    Extract locational data

In [0]:
# Feature selection --------------------------------------
# Extract dayofyear, weekofyear, month, dayofweek, hourofday (all data is from 2016)
sdf['day_of_year'] = sdf['pickup_datetime'].dt.dayofyear # (1 to 366)
sdf['week_of_year'] = sdf['pickup_datetime'].apply(lambda x: x.strftime('%W')).astype(int)
sdf['month'] = sdf['pickup_datetime'].dt.month
sdf['day_of_week'] = sdf['pickup_datetime'].dt.dayofweek  # Monday=0, Sunday=6
sdf['hour_of_day'] = sdf['pickup_datetime'].dt.hour
# we can drop datetime after extract trip_duration and features from pickup_datetime
sdf = sdf.drop(['dropoff_datetime','pickup_datetime','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','trip_duration'], axis=1)
display(sdf.head())
sdf.info()

id,vendor_id,passenger_count,store_and_fwd_flag,trip_duration_min,day_of_year,week_of_year,month,day_of_week,hour_of_day
1080784,2,1,N,6,60,9,2,0,16
889885,1,2,N,18,71,10,3,4,23
857912,2,2,N,27,52,7,2,6,17
3744273,2,6,N,19,5,1,1,1,9
232939,1,1,N,14,48,7,2,2,6


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 729322 entries, 0 to 729321
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   id                  729322 non-null  int64
 1   vendor_id           729322 non-null  <U0  
 2   passenger_count     729322 non-null  int64
 3   store_and_fwd_flag  729322 non-null  <U0  
 4   trip_duration_min   729322 non-null  int64
 5   day_of_year         729322 non-null  int64
 6   week_of_year        729322 non-null  int64
 7   month               729322 non-null  int64
 8   day_of_week         729322 non-null  int64
 9   hour_of_day         729322 non-null  int64
dtypes: int64(8), str(2)

In [0]:
import geopandas as gpd

geo_col = ['id', 'pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude']
geo_sp = spark.read.table("tripduration.default.raw_data")

'''geo_sp = (spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("raw_data")) #sdf.loc[:, geo_col]
'''

#filter only geo columns
geo_sp = geo_sp.select('id', 'pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude')
geo_df = geo_sp.toPandas()

geo_df = gpd.GeoDataFrame(geo_df)
geo_df['geometry_pickup'] = gpd.points_from_xy(geo_df.pickup_longitude, geo_df.pickup_latitude)
geo_df['geometry_dropoff'] = gpd.points_from_xy(geo_df.dropoff_longitude, geo_df.dropoff_latitude)
gdf_pickup = gpd.GeoDataFrame(geometry=geo_df['geometry_pickup'], crs="EPSG:4326") 
gdf_dropoff = gpd.GeoDataFrame(geometry=geo_df['geometry_dropoff'], crs="EPSG:4326")

# Project to World Mercator for metric distance calculations
gdf_pickup_metric = gdf_pickup.to_crs("EPSG:3395")
gdf_dropoff_metric = gdf_dropoff.to_crs("EPSG:3395")

# Calculate distances in meters
geo_df['distance_km'] = (gdf_pickup_metric.distance(gdf_dropoff_metric.geometry))/1000
geo_df = geo_df.drop(['geometry_pickup','geometry_dropoff'], axis=1)

geo_df['id'] = geo_df['id'].str.extract('(\d+)') # extract id from the trip id's number 
geo_df['id'] = geo_df['id'].astype('int64')

# conver to pyspark.pandas
geo_sdf = ps.from_pandas(geo_df)
geo_sdf.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 729322 entries, 0 to 729321
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 729322 non-null  int64  
 1   pickup_longitude   729322 non-null  float64
 2   pickup_latitude    729322 non-null  float64
 3   dropoff_longitude  729322 non-null  float64
 4   dropoff_latitude   729322 non-null  float64
 5   distance_km        729322 non-null  float64
dtypes: float64(5), int64(1)

In [0]:
# Join table and save table
sdf_join = sdf.merge(geo_sdf, on='id', how='inner')
display(sdf_join.head())
sdf_join.info()

id,vendor_id,passenger_count,store_and_fwd_flag,trip_duration_min,day_of_year,week_of_year,month,day_of_week,hour_of_day,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance_km
1,2,1,N,18,166,24,6,1,10,-74.01392364501955,40.71392440795898,-73.97164916992188,40.746406555175774,6.6887516432808205
5,2,1,N,6,116,17,4,0,9,-73.95310974121094,40.79858016967773,-73.96817779541014,40.800270080566406,1.6955368360772058
33,2,1,N,5,166,24,6,1,7,-73.9623794555664,40.75550842285156,-73.97299194335938,40.74930953979492,1.489646224884472
34,2,1,N,9,164,23,6,6,18,-73.98397827148438,40.76301956176758,-73.98597717285155,40.752288818359375,1.5866044365544167
43,2,1,N,20,178,25,6,6,0,-73.99559783935547,40.72676849365234,-73.94813537597656,40.789642333984375,10.613087703414163


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 729322 entries, 0 to 729321
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  int64  
 1   vendor_id           729322 non-null  object 
 2   passenger_count     729322 non-null  int64  
 3   store_and_fwd_flag  729322 non-null  object 
 4   trip_duration_min   729322 non-null  int64  
 5   day_of_year         729322 non-null  int64  
 6   week_of_year        729322 non-null  int64  
 7   month               729322 non-null  int64  
 8   day_of_week         729322 non-null  int64  
 9   hour_of_day         729322 non-null  int64  
 10  pickup_longitude    729322 non-null  float64
 11  pickup_latitude     729322 non-null  float64
 12  dropoff_longitude   729322 non-null  float64
 13  dropoff_latitude    729322 non-null  float64
 14  distance_km         729322 non-null  float64
dtypes: float64(5), int64(8), object

In [0]:
sdf_join['vendor_id'] = sdf_join['vendor_id'].astype('string')
sdf_join['store_and_fwd_flag'] = sdf_join['store_and_fwd_flag'].astype('string')
display(sdf_join.head())
sdf_join.info()


id,vendor_id,passenger_count,store_and_fwd_flag,trip_duration_min,day_of_year,week_of_year,month,day_of_week,hour_of_day,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance_km
1,2,1,N,18,166,24,6,1,10,-74.01392364501955,40.71392440795898,-73.97164916992188,40.746406555175774,6.6887516432808205
3,2,5,N,17,76,11,3,2,10,-73.99075317382812,40.7547607421875,-74.00711822509766,40.74153137207031,2.658670774502534
5,2,1,N,6,116,17,4,0,9,-73.95310974121094,40.79858016967773,-73.96817779541014,40.800270080566406,1.6955368360772058
8,1,1,N,5,167,24,6,2,9,-74.00962829589844,40.72476196289063,-74.015869140625,40.71548461914063,1.5248379676343982
16,1,1,N,6,89,13,3,1,16,-73.96031951904298,40.770320892333984,-73.94622039794923,40.77273559570313,1.6088401819560962


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 729322 entries, 0 to 729321
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  int64  
 1   vendor_id           729322 non-null  string 
 2   passenger_count     729322 non-null  int64  
 3   store_and_fwd_flag  729322 non-null  string 
 4   trip_duration_min   729322 non-null  int64  
 5   day_of_year         729322 non-null  int64  
 6   week_of_year        729322 non-null  int64  
 7   month               729322 non-null  int64  
 8   day_of_week         729322 non-null  int64  
 9   hour_of_day         729322 non-null  int64  
 10  pickup_longitude    729322 non-null  float64
 11  pickup_latitude     729322 non-null  float64
 12  dropoff_longitude   729322 non-null  float64
 13  dropoff_latitude    729322 non-null  float64
 14  distance_km         729322 non-null  float64
dtypes: float64(5), int64(8), string

In [0]:
spark_df = sdf_join.to_spark()
spark_df.write.format("delta").saveAsTable("tripduration.default.process_data") #.mode("overwrite")